## Twitter Pull using Twython
### Author: Kevin Mentzer
### Additions: Max Paul
### Last Update Date: 4-16-21
### adding SQL

In [1]:
from twython import Twython

twitter = Twython('5s2E2pi6FiF3aLwq76bowa99u',
'nCI8surLZmgkxuC3vCIfmWnMbqh4MJpNY7pycUmFlqyvdptfgS',
'444208254-ZtOSdwvZqAjwYqSQB9vXMUzAT6XMXmvbyTyj6dUO',
'h3UKqsfcjuRAgC0ut5jg3TF560nXurCeD0rWjwCmxYmH0')

#twitter.get_home_timeline()
#twitter.search(q='@NBATopShot+AND+Giveaway')

In [3]:
from twython import TwythonStreamer  
import csv
import codecs
import json
import time
# Filter out unwanted data
def process_tweet(tweet):  
    d = {}
    d['hashtags'] = [hashtag['text'] for hashtag in tweet['entities']['hashtags']]
    d['tweet_id'] = tweet['id']
    d['created_at'] = tweet['created_at']
    d['text'] = tweet['text']
    d['name'] = tweet['user']['name']
    d['user'] = tweet['user']['screen_name']
    d['user_id'] = tweet['user']['id']
    d['user_loc'] = tweet['user']['location']
    d['user_desc'] = tweet['user']['description']
    d['user_followers'] = tweet['user']['followers_count']
    d['user_friends'] = tweet['user']['friends_count']
    d['user_listed'] = tweet['user']['listed_count']
    d['user_created'] = tweet['user']['created_at']
    d['user_favs'] = tweet['user']['favourites_count']
    d['user_verified'] = tweet['user']['verified']
    d['user_statuses'] = tweet['user']['statuses_count']

    return d


# Create a class that inherits TwythonStreamer
class MyStreamer(TwythonStreamer):     

    # Received data
    def on_success(self, data):

        # Save full JSON to file
        with open('topGiveaway.json', 'w') as jsonfile:
            json.dump(data, jsonfile)        

        # Only collect tweets in English
        if data['lang'] == 'en':
            tweet_data = process_tweet(data)
            self.write_to_pgadmin(tweet_data)
            #would call mongo function here to write it to the database

    # Problem with the API
    def on_error(self, status_code, data):
        print(status_code, data)
        self.disconnect()

    # Save each tweet to csv file
    def save_to_csv(self, tweet):
        with open('topgiveaway.csv', 'a', encoding="utf8") as file:
            writer = csv.writer(file)
            writer.writerow(list(tweet.values()))
            
    def write_to_pgadmin(self,tweet):
        import psycopg2
        conn = psycopg2.connect(host="localhost",
                                database="top-twit-mapping",
                                port=5432,
                                user='postgres',
                                password=3301)
        cur = conn.cursor()
        tweet = tweet
        # insert user information
        command = '''INSERT INTO twitter_user (realname,username,user_id,user_loc,user_desc,user_followers,user_friends,user_listed,user_created,user_favs,user_verified,user_statuses) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s) ON CONFLICT
                     (user_id) DO NOTHING;'''
        cur.execute(command,(tweet['name'],tweet['user'],tweet['user_id'],tweet['user_loc'],tweet['user_desc'],tweet['user_followers'],tweet['user_friends'],tweet['user_listed'],tweet['user_created'],tweet['user_favs'],tweet['user_verified'],tweet['user_statuses']))     
       
        command2 = '''INSERT INTO twitter_tweet (tweet_id,created_at,tweet,user_id) VALUES (%s,%s,%s,%s) ON CONFLICT
                     (tweet_id) DO NOTHING;'''
        cur.execute(command2,(tweet['tweet_id'],tweet['created_at'],tweet['text'],tweet['user_id']))
        
        conn.commit()
        
        cur.close()
        
        conn.close()
        
while True:
    try:
        # Instantiate from our streaming class
        stream = MyStreamer('5s2E2pi6FiF3aLwq76bowa99u','nCI8surLZmgkxuC3vCIfmWnMbqh4MJpNY7pycUmFlqyvdptfgS','444208254-ZtOSdwvZqAjwYqSQB9vXMUzAT6XMXmvbyTyj6dUO','h3UKqsfcjuRAgC0ut5jg3TF560nXurCeD0rWjwCmxYmH0')

        # Start the stream
        stream.statuses.filter(track='@NBATopShot,Giveaway,TS:') #Track uses comma separated list 
    except (KeyboardInterrupt):
        print('Exiting')
        break
    except Exception as e:   
        print("error - sleeping " + str(e))
        time.sleep(5)
        continue


error - sleeping ('Connection broken: IncompleteRead(0 bytes read, 1 more expected)', IncompleteRead(0 bytes read, 1 more expected))
error - sleeping ('Connection broken: IncompleteRead(0 bytes read, 1 more expected)', IncompleteRead(0 bytes read, 1 more expected))
error - sleeping ('Connection broken: IncompleteRead(0 bytes read)', IncompleteRead(0 bytes read))
error - sleeping ('Connection broken: IncompleteRead(0 bytes read)', IncompleteRead(0 bytes read))
error - sleeping ('Connection broken: IncompleteRead(0 bytes read)', IncompleteRead(0 bytes read))
error - sleeping ('Connection broken: IncompleteRead(0 bytes read, 1 more expected)', IncompleteRead(0 bytes read, 1 more expected))
error - sleeping ('Connection broken: IncompleteRead(0 bytes read, 1 more expected)', IncompleteRead(0 bytes read, 1 more expected))
error - sleeping ('Connection broken: IncompleteRead(0 bytes read, 1 more expected)', IncompleteRead(0 bytes read, 1 more expected))
Exiting
